# Clustering redshifts with *yet_another_wizz*

This notebooks summarises the steps to compute clustering redshifts for an
unknown sample of galaxies using a reference sample with known redshifts.
Additionally, a correction for the galaxy bias of the reference sample is
applied (see Eqs. 17 & 20 in
[van den Busch et al. 2020](https://arxiv.org/pdf/2007.01846)).

This involves a number of steps:
1. Preparing the input data (creating randoms, applying masks; simplfied here).
2. Splitting the data into spatial patches and cache them on disk for faster access.
3. Computing the autocorrelation function amplitude $w_{\rm ss}(z)$, used as
   correction for the galaxy bias
4. Computing the cross-correlation function amplitude $w_{\rm sp}(z)$, which is
   the biased redshift estimate.
5. Summarising the result by correcting for the refernece sample bias and
   produce a redshift PDF.
6. Removing the cached data which is no longer needed.

At the very end, all stages are linked together to produce a `pipeline.yaml`
file for reference.

The aim of this notebook is to **give an overview of the wrapper functionality**,
including a summary of all currently implemented optional parameters (commented).
It is not meant to be a demonstaration of the performance of *yet_another_wizz*
since the example data used here is very small and the resulting signal-to-noise
ratio is quite poor.

In [ ]:
from rail.core.data import TableHandle

# configure RAIL datastore to allow overwriting data
from rail.core.stage import RailStage
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [ ]:
# step 1
from yaw import UniformRandoms
from rail.yaw_rail.utils import get_dc2_test_data

from rail.estimation.algos.cc_yaw import (
    stage_helper,  # utility for YawCacheCreate
    YawCacheCreate,     # step 2
    YawAutoCorrelate,   # step 3
    YawCrossCorrelate,  # step 4
    YawSummarize,       # step 5
    YawCacheDrop,       # step 6
)  # equivalent: from rail.yaw_rail import *

In [ ]:
VERBOSE = "debug"  # verbosity level of built-in logger, disable with "error"

## 1. Data preparation

Since this is a simple example, we are not going into the details of creating
realistic randoms and properly masking the reference and unknown data to a
shared footprint on sky. Instead, we are using a simulated dataset that serves
as both, reference and unknown sample.

First, we download the small test dataset derived from 25 sqdeg of DC2,
containing 100k objects on a limited redshift range of $0.2 < z < 1.8$. We add
the data as new handle to the datastore.

In [ ]:
test_data = get_dc2_test_data()  # downloads test data, cached for future calls
redshifts = test_data["z"].to_numpy()
zmin = redshifts.min()
zmax = redshifts.max()
n_data = len(test_data)
print(f"N={n_data}, {zmin:.1f}<z<{zmax:.1f}")

test_data_handle = DS.add_data("input_data", test_data, TableHandle)

Next we generate a x10 enhanced uniform random dataset for the test data
constrained to its rectangular footprint. We add redshifts by cloning the
redshift column `"z"` of the dataset.  We add the randoms as new handle to the
datastore.

In [ ]:
angular_rng = UniformRandoms(
    test_data["ra"].min(),
    test_data["ra"].max(),
    test_data["dec"].min(),
    test_data["dec"].max(),
    seed=12345,
)
test_rand = angular_rng.generate(n_data * 10, draw_from=dict(z=redshifts))

test_rand_handle = DS.add_data("input_rand", test_rand, TableHandle)

## 2. Splitting and caching the data

This step is crucial to compute consistent clustering redshift uncertainties.
*yet_another_wizz* uses spatial (jackknife) resampling and therefore, every
input dataset must be split into the same exact spatial regions/patches. To
improve the parallel performance, the datasets and randoms are pre-arranged into
these patches and cached on disk for better random patch-wise access. While this
is slow for small datasets, it is highly beneficial for large datasets with many
patches and/or memory constraints.

The RAIL wrapper uses manually specified cache directories, each of which contains
one dataset and optionally corresponding randoms. This ensures that the patch
centers are defined consistently. To create a new cache, use the
`YawCacheCreate.create()` method.

### Note on names and aliasing in RAIL

We need to create separate caches for the reference and the unknown data, which
means that we need to run the `YawCacheCreate` twice. Since that creates name
clashes in the RAIL datastore, we need to properly alias the inputs (`data`/
`rand`) and the output (`cache`) by providing a dictionary for the `aliases`
parameter when calling the `make_stage()`, e.g. by adding a unique suffix:

In [ ]:
name = "stage_name"
aliases = dict(data="data_suffix", rand="rand_suffix", cache="cache_suffix")

There is a shorthand for convenience (`from rail.yaw_rail.cache.AliasHelper`)
that allows to generate this dictionary by just providing a suffix name for the
stage instance (see example below).

In [ ]:
name = "stage_name"
aliases = stage_helper("suffix")

### The reference data

To create a cache directory we must specify a `path` to the directory at which the
data will be cached. This directory must not exist yet. We also have to specify
a `name` for the stage to ensure that the reference and unknown caches (see below)
are properly aliased to be distinguishable by the RAIL datastore.

Furthermore, a few basic column names that describe the tabular input data must
be provided. These are right ascension (`ra_name`) and declination (`dec_name`),
and in case of the reference sample also the redshifts (`redshift_name`).
Finally, the patches must be defined and there are three ways to do so:
1. `n_patches`: Generating a given number of patches from the object positions
   (peferrably of the randoms if possible) using k-means clustering.
2. `patch_name`: Providing a column name in the input table which contains patch
   indices (using 0-based indexing).
3. `patches`: Using the patch centers from a different cache instance, given by
   its path in the file system.

In [ ]:
stage_cache_ref = YawCacheCreate.make_stage(
    name="cache_ref",
    aliases=stage_helper("ref"),
    path="run/test_ref",
    overwrite=True,  # default: False
    ra_name="ra",
    dec_name="dec",
    redshift_name="z",
    # weight_name=,
    # patches=,
    # patch_name=,
    n_patches=5,
    verbose=VERBOSE,  # default: "info"
)
cache_ref = stage_cache_ref.create(
    data=test_data_handle,
    rand=test_rand_handle,
)

We can see from the log messages that *yet_another_wizz* processes the randoms
first and generates patch centers (`creating 5 patches`) and then applies them
to the dataset, which is processed last (`applying 5 patches`). Caching the data
can take considerable time depending on the hardware and the number of patches.

### The unknown data

The same procedure for the unknown sample, however there are some small, but
important differences. We use a different `path` and `name`, do not specify the
`redshift_name` (since we would not have this information with real data), and
here we chose to not provide any randoms for the unknown sample and instead rely
on the reference sample randoms for cross-correlation measurements.

Most importantly, we must provide the path to the reference data cache
(`patches="./test_ref"` instead of `n_patches`) since we must ensure that the
patch centers are identical. Even if the reference and unknown data are the same
as in this case, the automatically generated patch centers are not deterministic.
We can see in the log messages that the code looks up the catalog in
`./test_ref/rand` and reports `applying 5 patches`.

In [ ]:
stage_cache_unk = YawCacheCreate.make_stage(
    name="cache_unk",
    aliases=stage_helper("unk"),
    path="run/test_unk",
    overwrite=True,  # default: False
    ra_name="ra",
    dec_name="dec",
    # redshift_name=,
    # weight_name=,
    patches="run/test_ref",
    # patch_name=,
    # n_patches=,
    verbose=VERBOSE,  # default: "info"
)
cache_unk = stage_cache_unk.create(
    data=test_data_handle,
    # rand=None,
)

## 3. Computing the autocorrelation / bias correction

The bias correction is computed from the amplitude of the angular autocorrelation
function of the reference sample. The measurement parameters are the same as for
the cross-correlation amplitude measurement, so we can define all configuration
parameters once in a dictionary.

As a first step, we need to decide on which redshift bins we want to compute the
clustering redshifts. Here we choose the redshift limits of the reference data
(`zmin`/`zmax`) and, since the sample is small, only 8 bins (`zbin_num`) spaced
linearly in redshift (default `method="linear"`). Finally, we have to define the
physical scales in kpc (`rmin`/`rmax`, converted to angular separation at each
redshift) on which we measure the correlation amplitudes.


**Optional parameters:** Bins edges can alternatively specifed manually through
`zbins`. To apply scale dependent weights, e.g. $w \propto r^{-1}$, specify
the power-law exponent as`rweight=-1`. The parameter `rbin_num` specifies the
radial resolution (logarithmic) of the weights.

In [ ]:
corr_config = dict(
    rmin=100,   # in kpc
    rmax=1000,  # in kpc
    # rweight=None,
    # rbin_num=50,
    zmin=zmin,
    zmax=zmax,
    zbin_num=8,  # default: 30
    # method="linear",
    # zbins=np.linspace(zmin, zmax, zbin_num+1)
    # crosspatch=True,
    # thread_num=None,
    verbose=VERBOSE,  # default: "info"
)

We then measure the autocorrelation using the `YawAutoCorrelate.correlate()`
method, which takes a single parameter, the cache (handle) of the reference dataset.

In [ ]:
stage_autocorr = YawAutoCorrelate.make_stage(
    name="autocorr",
    **corr_config,
)
w_ss = stage_autocorr.correlate(
    sample=cache_ref,
)

As the code is progressing, we can observe the log messages of *yet_another_wizz*
which indicate the performed steps: getting the cached data, generating a pairs
of spatial patches to correlate, and counting pairs. Finally, the pair counts
are stored as custom data handle in the datastore.

We can interact with the returned pair counts these manually if we want to
investigate the results:

In [ ]:
counts_ss = w_ss.data  # extract payload from handle
counts_ss.dd

## 4. Computing the cross-correlation / redshift estimate

The cross-correlation amplitude, which is the biased estimate of the unknown
redshift distribution, is computed similarly to the autocorrelation above. We
measure the correlation using the `YawCrossCorrelate.correlate()` method, which
takes two parameters, the cache (handles) of the reference and the unknown data.

In [ ]:
stage_crosscorr = YawCrossCorrelate.make_stage(
    name="crosscorr",
    **corr_config,
)
w_sp = stage_crosscorr.correlate(
    reference=cache_ref,
    unknown=cache_unk,
)

As before, we can see the actions performed by *yet_another_wizz*. The main
difference for the cross-correlation function is that the second sample (the
unknown data/randoms) are not binned by redshift when counting pairs.

As before, we can interact with the result, e.g. by evaluating the correlation
estimator manually and getting the cross-correlation amplitude per redshift bin.

In [ ]:
counts_sp = w_sp.data  # extract payload from handle
corr_sp = counts_sp.sample()  # evaluate the correlation estimator
corr_sp.data

## 5. Computing the redshift PDF

The final analysis step is combining the two measured correlation amplitudes to
get a redshift estimate which is corrected for the reference sample bias and
converting it to a proper PDF. The latter is not trivial and different methods
have been tested in the literature to remove negative correlation amplitudes.
For simplicity, there is currently just one method implemented in the
*yet_another_wizz* wrapper, which sets all negative or non-finite values to
zero.

We use `YawSummarize.summarize()` method, which takes the pair count handles of
the cross- and autocorrelation functions as input. In principle, the
autocorrelation of the unknown sample could be specified to fully correct for
galaxy bias, however this is not possible in practice since the exact redshifts
of the unknown objects are not known.

All stage parameters are optional and usually not required. They can be used to
customise the correlation estimators used for each correlation function. They
default to Landy-Szalay (`"LS"`) where possible, otherwise Davis-Peebles (`"DP"`).

In [ ]:
stage_estimate = YawSummarize.make_stage(
    name="estimate",
    # cross_est=,
    # ref_est=,
    # unk_est=,
    # crosspatch=True,
    verbose=VERBOSE,  # default: "info"
)
estimate = stage_estimate.summarize(
    cross_corr=w_sp,
    ref_corr=w_ss,  # default: None
    # unk_corr=None,
)

The stage produces two outputs, a `qp.Ensemble`, tagged as `output` which
contains jackknife samples of the clustering redshift estimate will all negative
amplitudes set to zero, and the uncorrected redshift estimate, tagged as
`yaw_cc`. Below is a comparison plot of the true redshift histogram, the
raw estimate from *yet_another_wizz* and the first jackknife sample stored in
the `qp.Ensemble`:

In [ ]:
ensemble, yaw_cc = estimate["output"].data, estimate["yaw_cc"].data

ax = ensemble.plot(xlim=[zmin, zmax], label="qp, sample 1")
test_data.hist("z", bins=w_sp.data.edges, density=True, color="0.8", ax=ax, label="true")
yaw_cc.normalised().plot(label="yet_another_wizz")
ax.legend()

## 6. Remove caches

The cached datasets are not automatically removed, since the algorithm does not
know when they are no longer needed. Additionally, the reference data could be
resued for future runs, e.g. for different tomographic bins.

Since that is not the case here, we use the `YawCacheDrop.drop()` method to
delete the cached data.

In [ ]:
stage_drop_ref = YawCacheDrop.make_stage(name="drop_ref")
stage_drop_ref.drop(cache_ref)

stage_drop_unk  = YawCacheDrop.make_stage(name="drop_unk")
stage_drop_unk.drop(cache_unk)

## Create a pipeline file

To build a working `ceci` pipeline, it is important to give unique names to the
stages defined in the sections above. By running the stages we implicitly
connect the in- and outputs of the stages already. Therefore we only need to
create the pipeline from the existing stages.

In [ ]:
import ceci


pipe = ceci.Pipeline.interactive()
stages = [
    stage_cache_ref,
    stage_cache_unk,
    stage_autocorr, 
    stage_crosscorr,
    stage_estimate,
    stage_drop_ref,
    stage_drop_unk,
]
   
for stage_ in stages:
    pipe.add_stage(stage_)

Finally, we need to initialise the pipeline. Assuming that the inptus will be
present in the datas tore before running this pipeline, we can initialise the
pipeline with dummy values. The same applies to the unused, optional stage
inputs `rand_unk` and `unk_corr`.

Finally, we save the pipeline and read it back for verification.

In [ ]:
pipe.initialize(
    dict(
        input_data="dummy.in",
        input_rand="dummy.in",
        rand_unk="/dev/null",
        unk_corr="/dev/null",
    ), 
    dict(
        output_dir=".",
        log_dir=".",
        resume=False
    ), 
    None,
)
pipe.save("pipeline.yml")
ceci.Pipeline.read("pipeline.yml")